In [1]:
import pandas as pd
import plotly.express as px

mapbox_token = "pk.eyJ1IjoiemhvbmdqdW5tYSIsImEiOiJja2pwbzA3c2YwNjd3MnJxaDFrcHh5NmNyIn0.6mzTX-clHuP5h9lUO17TCw"
BORDER_SIZE = 10
DATA_DIR = '../blog-data'

# Congestion of Ports

We define two congestion indicators: 

(1) Average Congestion Time (ACT), which is the average waiting time of a ship at the anchorage before it berths at the terminal, 

\begin{align}
\text{ACT} = \frac{\text{Total congestion time of this month}}{\text{Total number of ships visiting this port during this month}}
\end{align}

(2) Average Congestion Rate (ACR), which is the ratio of ships that moor at anchorage before berthing at the terminal over the total number of ship visits at a port,

\begin{align}
\text{ACR} = \frac{\text{Total number of ships suffering from congestion in this port during this month}}{\text{Total number of ships visiting this port during this month}}
\end{align}

The related data can be downloaded by clicking the following links. This Excel file contains the monthly aggregated congestion indices of top container ports since 2017:

- [Aggregated Congestion Indices](./data/congestion_indices.xlsx)

## IMA-DBSCAN

In the study of Bai *et al*. {cite}`baiDataDrivenIterativeMultiAttribute2023a`, the authors proposed an berth detection algorithm, which is called an iterative, multi-attribute DBSCAN (IMA-DBSCAN). The algorithm is applied to measure congestion at 20 major container ports in the world.

## Congestion in Panama Canal

Beginning in October 2023, rainfall near the Panama Canal was 41% less than usual, making it the driest month since the earliest registers. As a result of the El Niño phenomenon, water availability in the reservoir system has been reduced and the Panama Canal Authority has announced a reduction in the number of reservation slots, which will reduce the capacity of the canal and cause congestion {cite}`rodriguezDriestMonthOctober2023`.

We tracked the vessel positions around the Panama Canal from early July 2023 to the present (2023-11).

In [2]:
df = pd.read_csv(f'{DATA_DIR}/panama_daily.csv', sep='|', header=0)

def plot_points(df):
    fig = px.scatter_mapbox(
        df,
        lat=df["latitude"],
        lon=df["longitude"],
        # hover_data=['imo', 'timestamp', 'speed', 'heading'],
        animation_frame="Date",
        zoom=7.5,
        title="Vessel Positions at Panama Canal from 2023-07",
        # width=800,
        # height=600,
    )
    fig.update_layout(
        mapbox={
            "accesstoken": mapbox_token,
            "style": "satellite-streets",
        },
        # margin={"l": BORDER_SIZE, "r": BORDER_SIZE},
    )
    return fig


fig = plot_points(df)
fig

## Visualization of Top Port Congestion

Here, we plot the monthly congestion index for the top 47 container ports since 2017.

In [3]:
df = pd.read_csv("../blog-data/congestion_new_with_coords.csv", sep="|", header=0)
df = df.loc[df["year"] >= 2017].copy()
df = df.round({"total_congestion_time": 1, "act": 2, "aact": 2, "acr": 4})
df = df.rename(columns={"act": "ACT", "aact": "AACT", "acr": "ACR"})
df["ACR"] = df["ACR"].apply(lambda x: format(x, ".2%"))
df["Time"] = df.apply(
    lambda x: pd.Timestamp(x["year"], x["month"], 1)
    .date()
    .strftime("%Y-%m"),
    axis=1,
)
sizeref = 2. * df['ACT'].max() / (40 ** 2)
df['size'] = df["ACT"]/sizeref

In [4]:
df

,port_name,year,month,num_of_ship,num_of_delay,total_congestion_time,ACT,AACT,ACR,longitude,latitude,Time,size
1,Algeciras,2017,1,247,66,1477.9,5.98,22.39,26.72%,-5.436667,36.126111,2017-01,25.701085
2,Algeciras,2017,2,193,45,1526.6,7.91,33.92,23.32%,-5.436667,36.126111,2017-02,33.995917
3,Algeciras,2017,3,223,27,603.2,2.71,22.34,12.11%,-5.436667,36.126111,2017-03,11.647147
4,Algeciras,2017,4,249,28,692.7,2.78,24.74,11.24%,-5.436667,36.126111,2017-04,11.947996
5,Algeciras,2017,5,288,73,1756.4,6.10,24.06,25.35%,-5.436667,36.126111,2017-05,26.216826
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,Yingkou,2023,6,39,16,401.7,10.30,25.11,41.03%,122.233333,40.683333,2023-06,44.267755
3896,Yingkou,2023,7,39,17,273.1,7.00,16.07,43.59%,122.233333,40.683333,2023-07,30.084882
3897,Yingkou,2023,8,36,13,163.0,4.53,12.54,36.11%,122.233333,40.683333,2023-08,19.469217
3898,Yingkou,2023,9,36,18,249.3,6.93,13.85,50.00%,122.233333,40.683333,2023-09,29.784034


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3854 entries, 1 to 3899
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   port_name              3854 non-null   object 
 1   year                   3854 non-null   int64  
 2   month                  3854 non-null   int64  
 3   num_of_ship            3854 non-null   int64  
 4   num_of_delay           3854 non-null   int64  
 5   total_congestion_time  3854 non-null   float64
 6   ACT                    3854 non-null   float64
 7   AACT                   3785 non-null   float64
 8   ACR                    3854 non-null   object 
 9   longitude              3854 non-null   float64
 10  latitude               3854 non-null   float64
 11  Time                   3854 non-null   object 
 12  size                   3854 non-null   float64
dtypes: float64(6), int64(4), object(3)
memory usage: 421.5+ KB


In [6]:
fig = px.scatter_geo(
    df,
    lat="latitude",
    lon="longitude",
    # text=congestion_df['act'].round(1).astype(int),
    size="size",
    hover_name="port_name",
    hover_data={
        "ACR": True,
        "ACT": True,
        "AACT": True,
        "Time": False,
        "size": False,
        "latitude": False,
        "longitude": False,
    },
    animation_frame="Time",
    projection="natural earth",
    # scope='asia',
    # width=800,
    # height=600,
    opacity=1,
    size_max=40,
    title="Congestion of top 50 container ports from 2017-01 to 2023-10",
)
# fig.update_layout(
#     margin={"l": BORDER_SIZE, "r": BORDER_SIZE},
# )
fig.show()

## References
```{bibliography}
:filter: docname in docnames
```